In [1]:
from theano.sandbox import cuda

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)


In [2]:
%matplotlib inline
#set FIXED variables
HOMEPATH = "/home/ubuntu/fastai/"
MODELPATH = HOMEPATH + 'dogscats/models/'
TRAINPATH = HOMEPATH + 'dogscats/train/'
VALIDPATH = HOMEPATH + 'dogscats/valid/'
MODELPATH = HOMEPATH + 'dogscats/models/'
TESTPATH = HOMEPATH + 'dogscats/test1/'
import os
from datetime import datetime


In [3]:
os.chdir(HOMEPATH)
print (os.getcwd())
import utils; reload(utils)
from utils import *
from __future__ import division, print_function
#NB: lecture shows as using Theano backend.

/home/ubuntu/fastai


Using TensorFlow backend.


In [4]:
from keras import backend as K
#import os#already imported

def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend

set_keras_backend("theano")

Using Theano backend.


In [5]:

#this is redundant since already generated this dir
if not os.path.exists(MODELPATH): os.mkdir(MODELPATH)

batch_size=64

In [6]:
model = vgg_ft(2)
#uses def vgg_ft(out_dim): from utils.py
#creates Vgg16 object &  calls Vgg16.ft(self, num) method
#Replace the last layer of the model with a Dense (fully connected) layer of 2 neurons.
#also lock the weights of all layers except the new layer so that we only learn
#weights for the last layer in subsequent training.

In [7]:
type(model)

keras.models.Sequential

In [8]:
print(len(model.layers))

38


In [9]:
model.load_weights(MODELPATH+'finetune3.h5')

In [10]:
layers = model.layers
print (type(layers), len(layers))

<type 'list'> 38


In [11]:
last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Convolution2D][-1]

In [12]:
last_conv_idx

30

In [13]:
layers[last_conv_idx]

In [14]:
conv_layers = layers[:last_conv_idx+1]  #all layers up to the last convolutional layer
#NB: none of the convolutional layers in vgg16 have dropout

conv_model = Sequential(conv_layers)
# Dense layers - also known as fully connected or 'FC' layers
fc_layers = layers[last_conv_idx+1:]

In [15]:
print (type(conv_model))
print (type(model))

<class 'keras.models.Sequential'>
<class 'keras.models.Sequential'>


In [16]:
print (type(conv_layers), len(conv_layers), type(conv_layers[0]))
print (type(conv_model))
print (type(fc_layers), len(fc_layers), type(fc_layers[0]))
i=0
for layer in conv_layers:
    print (i, type(layer))
    i += 1

<type 'list'> 31 <class 'keras.layers.core.Lambda'>
<class 'keras.models.Sequential'>
<type 'list'> 7 <class 'keras.layers.pooling.MaxPooling2D'>
0 <class 'keras.layers.core.Lambda'>
1 <class 'keras.layers.convolutional.ZeroPadding2D'>
2 <class 'keras.layers.convolutional.Convolution2D'>
3 <class 'keras.layers.convolutional.ZeroPadding2D'>
4 <class 'keras.layers.convolutional.Convolution2D'>
5 <class 'keras.layers.pooling.MaxPooling2D'>
6 <class 'keras.layers.convolutional.ZeroPadding2D'>
7 <class 'keras.layers.convolutional.Convolution2D'>
8 <class 'keras.layers.convolutional.ZeroPadding2D'>
9 <class 'keras.layers.convolutional.Convolution2D'>
10 <class 'keras.layers.pooling.MaxPooling2D'>
11 <class 'keras.layers.convolutional.ZeroPadding2D'>
12 <class 'keras.layers.convolutional.Convolution2D'>
13 <class 'keras.layers.convolutional.ZeroPadding2D'>
14 <class 'keras.layers.convolutional.Convolution2D'>
15 <class 'keras.layers.convolutional.ZeroPadding2D'>
16 <class 'keras.layers.convol

In [ ]:
batches = get_batches(TRAINPATH, shuffle=False, batch_size=batch_size)
val_batches = get_batches(VALIDPATH, shuffle=False, batch_size=batch_size)

val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [ ]:
batches.class_indices

In [ ]:
startTime= datetime.now()
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

print (type(val_features), val_features.shape)

In [ ]:
startTime= datetime.now()
trn_features = conv_model.predict_generator(batches, batches.nb_sample)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

print (type(trn_features), trn_features.shape)

In [ ]:
save_array(MODELPATH + 'train_convlayer_features.bc', trn_features)
save_array(MODELPATH + 'valid_convlayer_features.bc', val_features)

In [ ]:
print ("trn_features:", type(trn_features), trn_features.shape)
print ("val_features:", type(val_features), val_features.shape)

In [ ]:
trn_features = load_array(MODELPATH+'train_convlayer_features.bc')
val_features = load_array(MODELPATH+'valid_convlayer_features.bc')

In [ ]:
trn_features.shape

In [ ]:
# Copy the weights from the pre-trained model.
# NB: Since we're removing dropout, we want to half the weights
# NBB: previously dropout was 0.5, so with no dropout we need to mult weights by 1/2
def proc_wgts(layer): 
    return [o/2 for o in layer.get_weights()]

In [ ]:
# Such a finely tuned model needs to be updated very slowly!
opt = RMSprop(lr=0.00001, rho=0.7)

In [ ]:
def get_fc_model():
    model = Sequential([
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.),
        Dense(4096, activation='relu'),
        Dropout(0.),
        Dense(2, activation='softmax')
        ])

    for l1,l2 in zip(model.layers, fc_layers): 
        l1.set_weights(proc_wgts(l2))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
fc_model = get_fc_model()

In [ ]:
type(fc_model)

In [ ]:
startTime= datetime.now()
fc_model.fit(trn_features, trn_labels, nb_epoch=8, 
             batch_size=batch_size, validation_data=(val_features, val_labels))
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

#nb: accuracy on training is higher than accuracy on validation. _OVERFITTING_
#week 3 video @ 1hr26'

#best accuracy training: 0.9909, validation: 0.9840

In [ ]:
#fc_model.save_weights(MODELPATH+'no_dropout.h5')#already saved

In [ ]:
#fc_model.load_weights(MODELPATH+'no_dropout.h5')#already have values in memory

In [ ]:
#data augmentation

# dim_ordering='tf' uses tensorflow dimension ordering,
#   which is the same order as matplotlib uses for display.
# Therefore when just using for display purposes, this is more convenient
gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1, 
       height_shift_range=0.1, shear_range=0.15, zoom_range=0.1, 
       channel_shift_range=10., horizontal_flip=True, dim_ordering='tf')

In [ ]:
# Create a 'batch' of a single image
img = np.expand_dims(ndimage.imread(HOMEPATH+'dogscats/test1/7.jpg'),0)
# Request the generator to create batches from this image
aug_iter = gen.flow(img)

In [ ]:
temp = ndimage.imread(HOMEPATH+'dogscats/test1/7.jpg')
print (type(temp), temp.shape)
print (type(img), img.shape)
print (type(aug_iter))

In [ ]:
# Get eight examples of these augmented images
aug_imgs = [next(aug_iter)[0].astype(np.uint8) for i in range(8)]
print (type(aug_imgs), len(aug_imgs), type(aug_imgs[0]), aug_imgs[0].shape)

In [ ]:
# The original
plt.imshow(img[0])

In [ ]:
# Augmented data
plots(aug_imgs, (20,7), 2)

In [ ]:
# Ensure that we return to theano dimension ordering
K.set_image_dim_ordering('th')
print (type(K))

In [ ]:
#adding augmentation
gen = image.ImageDataGenerator(rotation_range=15, 
                               width_shift_range=0.1, 
                               height_shift_range=0.1, 
                               zoom_range=0.1, 
                               horizontal_flip=True)
print (type(gen))

In [ ]:
batches = get_batches(TRAINPATH, gen, batch_size=batch_size)

#from vgg16.py
#def get_batches(self, path, gen=image.ImageDataGenerator(), shuffle=True, batch_size=8, class_mode='categorical'):
# NB: We don't want to augment or shuffle the validation set

val_batches = get_batches(VALIDPATH, shuffle=False, batch_size=batch_size)

#NB: output reports the same number of images belonging to same number of classes as when 
#keras.preprocessing.image.ImageDataGenerator was not used.

print ("batches:", type(batches))
print ("val_batches:", type(val_batches))

In [ ]:
print (len(conv_model.layers))
fc_model = get_fc_model()
print(len(fc_model.layers))

In [ ]:
#set convolutions models to non trainable
for layer in conv_model.layers: 
    layer.trainable = False
# Look how easy it is to connect two models together!
conv_model.add(fc_model)

In [ ]:
print(len(conv_model.layers))

In [ ]:
#show the layers
i=0
for layer in conv_model.layers:
    print (i, type(layer))
    i += 1

In [ ]:
startTime= datetime.now()
conv_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


In [ ]:
startTime= datetime.now()
conv_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=8, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

In [ ]:
#summary from above
#best training accuracy   :0.9833
#best validation accuracy :0.9810

#identical - diff number of epochs
#this takes a long time to run.
print ("start")
startTime= datetime.now()
conv_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=3, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

#summary for this block
#best training accuracy   :0.9852
#best validation accuracy :0.9820
#if training accuracy > validation accuracy then overfitting (maybe)

In [ ]:
conv_model.save_weights(MODELPATH + 'aug1.h5')


In [ ]:
#conv_model.load_weights(MODELPATH + 'aug1.h5') #model already loaded.
#print ("model loaded")

In [ ]:
#Batch normalization (batchnorm) 

In [ ]:
print (type(conv_layers), len(conv_layers), type(conv_layers[0]))
print ("last layer :", type(conv_layers[-1]))
print (conv_layers[-1].output_shape[1:])

i=0
for layer in conv_layers:
    print (i, layer.output_shape[1:])
    i += 1

In [ ]:
def get_bn_layers(p):
    '''
    creates layers with sequence of layers as coded.
    dropout rate as specified
    '''
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(1000, activation='softmax')
        ]

In [ ]:
def load_fc_weights_from_vgg16bn(model):
    "Load weights for model from the dense layers of the Vgg16BN model."
    # See imagenet_batchnorm.ipynb for info on how the weights for
    # Vgg16BN can be generated from the standard Vgg16 weights.
    from vgg16bn import Vgg16BN
    vgg16_bn = Vgg16BN()
    _, fc_layers = split_at(vgg16_bn.model, Convolution2D)
    copy_weights(fc_layers, model.layers)

In [ ]:
p=0.6

In [ ]:
temp = get_bn_layers(0.6)
print (type(temp), len(temp), type(temp[0]))

In [ ]:
bn_model = Sequential(get_bn_layers(0.6))

In [ ]:
print (type(bn_model), len(bn_model.layers))

In [ ]:
load_fc_weights_from_vgg16bn(bn_model)

In [ ]:
def proc_wgts(layer, prev_p, new_p):
    scal = (1-prev_p)/(1-new_p)
    return [o*scal for o in layer.get_weights()]

In [ ]:
temp = proc_wgts(l, 0.5, 0.6)
print (type(temp), len(temp))
print (type(temp[0]), type(temp[1]))
print (temp[0].shape, temp[1].shape)
print ((1-0.5)/(1-0.6))

In [ ]:
#for each layer in bn_model.layers
#set weights ine ach layer to a scalar mult of existing weights.
#avoid running this multiple times!!!!

for l in bn_model.layers: 
    if type(l)==Dense: l.set_weights(proc_wgts(l, 0.5, 0.6))

In [ ]:
print (type(bn_model), type(bn_model.layers), len(bn_model.layers))
i=0
for layer in bn_model.layers:
    print (i, type(layer), layer.output_shape[1:])
    i += 1

In [ ]:
bn_model.pop()
#remove last layer

In [ ]:
print (type(bn_model), type(bn_model.layers), len(bn_model.layers))

i=0
for layer in bn_model.layers:
    print (i, type(layer), layer.trainable, layer.output_shape[1:])
    i += 1

In [ ]:
#set all layers to non trainable
for layer in bn_model.layers: 
    layer.trainable=False

In [ ]:
#add Dense layer back - we deleted Dense layer during "bn_model.pop()"
bn_model.add(Dense(2,activation='softmax'))

In [ ]:
i=0
for layer in bn_model.layers:
    print (i, type(layer), layer.trainable, layer.output_shape[1:])
    i += 1

In [ ]:
type(bn_model)
#https://keras.io/models/sequential/
#compile(self, optimizer, loss, metrics=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
#https://keras.io/optimizers/
#Adam - A method for stochastic optimisation : https://arxiv.org/abs/1412.6980v8

In [ ]:
print ("start")
startTime= datetime.now()
bn_model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


In [ ]:
print ("start")
startTime= datetime.now()
bn_model.fit(trn_features, trn_labels, nb_epoch=8, validation_data=(val_features, val_labels))
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))
#this will be fast since only retraining last layer
#accuracy: training:0.9658 validation:0.9800 not overtrained, accuracy could be higher.

In [ ]:
bn_model.save_weights(MODELPATH+'bn.h5')

In [ ]:
#bn_model.load_weights(model_path+'bn.h5')#value already in memory.

In [ ]:
bn_layers = get_bn_layers(0.6)
print (type(bn_layers), len(bn_layers), type(bn_layers[0]))


In [ ]:
bn_layers.pop()

In [ ]:
#check # of layers after pop
print (type(bn_layers), len(bn_layers), type(bn_layers[0]))

In [ ]:
#replace last layer just deleted. same specs as previous layer added.
bn_layers.append(Dense(2,activation='softmax'))

In [ ]:
#check # of layers after adding layer
print (type(bn_layers), len(bn_layers), type(bn_layers[0]))

In [ ]:
final_model = Sequential(conv_layers)
print (type(final_model), type(final_model.layers), len(final_model.layers), type(final_model.layers[0]))

for layer in final_model.layers: 
    layer.trainable = False

for layer in bn_layers: 
    final_model.add(layer)

print (type(final_model), type(final_model.layers), len(final_model.layers), type(final_model.layers[0]))

In [ ]:
#print (type(final_model), type(final_model.layers), len(final_model.layers), type(final_model.layers[0]))
#temp cell to show 31+9=40 layers

In [ ]:
print(type(bn_model.layers))
print(type(bn_layers))

In [ ]:
#why are we copying weights from bn_layers to bn_model.layers?
for l1,l2 in zip(bn_model.layers, bn_layers):
    l2.set_weights(l1.get_weights())

In [ ]:
final_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
startTime= datetime.now()
final_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))
#accuracy - train:0.9928  validation:0.9710  - slight overtraining. 

In [ ]:
final_model.save_weights(MODELPATH + 'final1.h5')

In [ ]:
startTime= datetime.now()
final_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=4, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))
#accuracy: training:0.9657 validation:0.9810

In [ ]:
final_model.save_weights(MODELPATH + 'final2.h5')

In [ ]:
final_model.optimizer.lr=0.001

In [ ]:
startTime= datetime.now()
final_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=4, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))
#accuracy : train:0.9673    validation:0.9795

In [ ]:
bn_model.save_weights(MODELPATH + 'final3.h5')

In [ ]:
print (type(bn_model))
batch_size

In [ ]:
print ("vgg.test start")
batches, preds = bn_model.predict_classes(TESTPATH, batch_size = batch_size*2)
print("type(batches):", type(batches))
print("type(preds):", type(preds))

In [ ]:
vgg = Vgg16()
print (type(vgg))
type(vgg.model)
